<a href="https://colab.research.google.com/github/LironSimon/Exploring_parameter_relations_from_trajectories/blob/main/add_info_to_trajectory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Issues
*   add vt2_in_salt values. **Don't run this notebook before that!**
*   Vy min is exctly that! greatly effected by noise
*   **fix csv 0001_1_1/2 in 0602_batch_1 before running**





### Import environment and setup

In [ ]:
# Import the necessary packages
import glob
import pandas as pd
import sys 
# from scipy.optimize import least_squares
import numpy as np
from sympy import symbols, solve

In [ ]:
# mount to drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# set paths
general_path = '/content/gdrive/Shareddrives/Liron_Simon_Thesis/Experimental_data/Trajectories'
notebook_path = f'{general_path}/experimental_notebook_summary.csv'
joint_data_path = f'{general_path}/joint_trajectory_data.csv'

In [ ]:
# !ls /content/gdrive/Shareddrives/Liron_Simon_Thesis/Experimental_data/Trajectories

In [ ]:
# set avg terminal velocity in salt water, as collected from experiment
# keys - rho2 density [gr/cm3], values - terminal veocity [m/s]
vt2_in_salt = {1.04: float("nan"), 
               1.05: float("nan"), 
               1.06: float("nan"), 
               1.07: float("nan"), 
               1.08: float("nan"), 
               1.09: float("nan"), 
               1.10: float("nan"), 
               1.11: float("nan"), 
               1.12: float("nan")}

### Write helpfull funcs for calculating data

1) finding terminal velocity acourding to trajectory

In [ ]:
def vt_from_data(V, pixels, layer, percent=0.10, num_of_frames=10):
    '''Takes a column of Vy values, coresponding pixel column, percent of change, and num of frames to avg. 
    Parameter 'layer' takes an indicator 1 or 2, that states whther to find values in the beginning or ending of the trajectory
    Returns: 1) the index at which V has change by over a defined percent from the avg of the first num_of_frames.
             2) the pixel detected.
             3) the avg value of the velocity up until that index.'''
    V = V.dropna()

    # if bottom layer is chosen, change order of columns
    if layer is 2: 
        V = V.iloc[::-1]
        pixels = pixels.iloc[::-1]

    # cut initial nan value that's in index 0
    else: V = V[1:]
    
    # create a list of indices of V deviations
    indices = []

    init = V[:num_of_frames].mean()
    for i,v in enumerate(V):
        if init*(1-percent) < v < init*(1+percent): continue
        
        else: 
            # ignore a single point if its alot diff from prior frame
            if i>0 and V[i-1]*0.5> v or v > V[i-1]*1.5: 
                indices.append(i)
                continue
            
            # change to real index if layer=2
            if layer is 2: 
                i = len(V)-i
                indices = [len(V)-j for j in indices]

            # calc mean up until i without deviations
            V = V.drop(indices)
            mean = V[:i].mean()

            return i, round(pixels.loc[i]), mean

2) Finding terminal velocity acourding to given parameters

In [ ]:
def Cd(Re):
    #if Re <= 0: Re = 1e-3   
    return 0.4 + 24.0/Re + 6.0/(1+Re**0.5)

def Vp(d): return np.pi*d**3/6
def Ap(d): return np.pi*d**2/4

def terminal_velocity(rhop=2500, rhof=1000 , g=9.81, d=1e-3, nu=1e-6):
    """ Inputs:
          rhop : particle density, (kg/m^3)
          rhof : fluid density, (kg/m^3)
          g : gravity acceleration, default = 9.81 ( m/s^2 )
          d : particle diameter, (m)
          nu : kinematic viscosity, (m^2/s), water is 1e-6   
    Outputs:
          sol : terminal velocity of the particle (m/s)
    """
    V = symbols('V')
    eq = (rhop-rhof)*Vp(d)*g - 0.5*Cd(V*d/nu)*rhof*Ap(d)*(V**2)
    sol = solve(eq)

    if len(sol)!=1: raise ValueError('more/less than one result for Vt')

    return sol[0]

3) Getting diameter from observed terminal velocity in upper layer

In [ ]:
def diam_from_terminal_velocity(vt=0.145, rhop=2500, rhof=1000, g=9.81, nu=1e-6):
    """ Inputs:
          vt1 : observed terminal velocity in upper fluid layer, (m/s)
          rhop : particle density, (kg/m^3)
          rhof : fluid density, (kg/m^3)
          g : gravity acceleration, default = 9.81 ( m/s^2 )
          nu : kinematic viscosity, (m^2/s), water is 1e-6   
    Outputs:
          d : diam (m)
    """
    d =  symbols('d')
    eq = (rhop-rhof)*Vp(d)*g - 0.5*Cd(vt*d/nu)*rhof*Ap(d)*(vt**2)
    sol = solve(eq)

    if len(sol)!=1: raise ValueError('more than one result for Vt')

    # make sure d is within manufacture's range
    # if 1e-3 > sol[0] or sol[0] > 1.18e-3: raise ValueError('d is not compatible to range.')

    return sol[0]

4) Calc interface thickness by enty and exit y pixel

In [ ]:
def h_in_meters(entry_pt, Batch_id, pixel2m= 0.088/1024):
      ####!!!!! Should change this to load from saved csv with all info if we decide to use laser pics
      # create a df with laser exit points
      df = pd.DataFrame(data=[429,417,404,416], index = [1,2,3,4])

      exit_pxl = int(df.loc[Batch_id])

      return (exit_pxl - entry_pt)*pixel2m, exit_pxl

5) reduce size of a df by storing vectors as a list in a single cell

In [ ]:
def columns2cells(df):
    for column in df.columns:
        if len(df[column].unique()) > 2:
            df[column] = df[column].astype('object')
            df.at[0,column] = df[column].tolist()
    return df.loc[0,:]

### Load csv's and add data to them
Also saves all csvs in a joint file for easier future analisys

get data from experimental notebook

In [ ]:
notebook = pd.read_csv(notebook_path)

loop over individual csv's and add data to them

In [ ]:
for i in range(len(notebook)):
    # extract data relevant to all csv's in the batch
    batch_id, rho_particle, rho_up, rho_down, density_ratio, fps, res, temp = notebook.iloc[i,1:-1]
    nu_up, nu_down = 1e-6, 1e-6
    date = notebook.at[i, 'date'][1:]

    # figure out which folder the data is relevent for
    folder_name = f'{date}_batch_{batch_id}'

    # make sure we have such a folder before we try and open it
    if folder_name not in ['0602_batch_1','0602_batch_2','0602_batch_3','0602_batch_4']: continue

    # loop over csvs in that folder, add info to them and load them to a joint file
    for csv_path in glob.iglob(f'{general_path}/{folder_name}/*'):
        df = pd.read_csv(csv_path)

        # delete unnecery columns that may apear
        if 'Unnamed: 0' in df.columns: df.pop('Unnamed: 0')

        # make sure this csv wan't updated before
        if len(df.columns) > 7 : continue

        # add/rewrite time tracking that starts from 0
        df['sec'] = [i/fps for i in range(len(df))]

        # add cells of info taken from notebook
        df.at[0,['rho_particle', 'rho_up', 'rho_down', 'density_ratio','nu_up','nu_down', 'fps', 'temp']] = rho_particle, rho_up, rho_down, density_ratio, nu_up, nu_down, fps, temp

        # use funcs to find Vt1 (up) and Vt2 (down) from velocity vector
        indx1, entry_y, Vt1 = vt_from_data(df.Vy,df.y,1)
        indx2, entr_y2, Vt2 = vt_from_data(df.Vy,df.y,2)

        # add cells with relevent parameter:
        df.at[0,['Vt1_observed','Vt2_observed']] = Vt1, Vt2                # terminal velocity in lower/upper layer
        d = diam_from_terminal_velocity(vt=Vt1, rhof=rho_up)               # particle diameter corresponding to the observes Vt1. Supposed to be in range of 1-1.18mm
        df.at[0,'d'] = d
        df.at[0,'Vt1_expected'] = terminal_velocity(rhof=rho_up,d=d)       # expected terminal velocity in upper layer given d
        df.at[0,'Vt2_expected'] = terminal_velocity(rhof=rho_down,d=d)     # expected terminal velocity in lower layer given d
        df.at[0,'Vt2_avg_from_exp'] = vt2_in_salt[round(rho_down/1000,2)]  # expected terminal velocity in lower layer given fluid density
        
        # calc and add Re values - 1: up, 2: down
        df.at[0,'Re1'], df.at[0,'Re2'] = Vt1*d/nu_up, Vt2*d/nu_down        # Reynolds number in upper/lower layer

        # calc and add Fr values - 1: up, 2: down
        h, exit_y = h_in_meters(entry_pt=entry_y, Batch_id=batch_id, pixel2m = res)
        N  = (2*9.81*(rho_down - rho_up)/h/(rho_up + rho_down))**0.5       # buoyancy frequency   [1/s]
        df.at[0,['Fr1','Fr2']] = [Vt1/(N*d), Vt2/(N*d)]                    # Froude number in upper/lower layer

        # get index of exit_Y
        indx3 = df.y.iloc[(df.y-exit_y).abs().argsort()[:1]].index.values[0]

        # get average velocity in interface
        df.at[0,'Vavg_in_h'] = df.loc[indx1:indx3,'Vy'].mean()

        # get tau recovery time - time from first change in velocity to reaching vt2
        df.at[0,'tau_recovery'] = df.loc[indx2, 'sec'] - df.loc[indx1, 'sec']

        ###### ??? SHOULD I GET MIN VALUE WITH GREATER CARE? ??? ########
        df.at[0,'Vy_min'] =  df.Vy.dropna().min()

        # get average velocity in the tau range
        df.at[0,'Vavg_in_tau'] = df.loc[indx1:indx2,'Vy'].mean() 

        # save updated df to drive
        df.to_csv(csv_path)

        ### save to a joint dataset
        # add identification
        df.at[0,['date','batch','file_id']] = date, batch_id, csv_path[-12:-4] 
        # turn df to a row
        df = columns2cells(df)                                                 
        df.to_csv(joint_data_path, mode='a')

show last csv that was saved

In [ ]:
df.to_csv(csv_path)